<a href="https://colab.research.google.com/github/alondraortegaherrera/coloreado_grafos/blob/main/coloreado_de_grafos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\color{DarkSlateGray}{\huge\text{Coloreado de Grafos con Recocido Simulado}}$

In [651]:
# @title **Librerías**
import networkx as nx
import matplotlib.pyplot as plt
import random
from random import shuffle, sample
import math

In [673]:
# @title **Problema**

"""
    El problema consiste en encontrar una asignación de colores para los vértices de un grafo de acuerdo con una
    secuencia de grados dada, con el objetivo de minimizar los conflictos entre los vértices adyacentes.
    Cada vértice debe ser asignado un color de un conjunto limitado de colores (en este caso, hasta 4 colores),
    de forma que los vértices adyacentes no compartan el mismo color.
"""

class ColoreadoGrafo():
    def __init__(self, degree_seq, num_min_colores=4):
        self.num_min_colores = num_min_colores
        self.grafo = self.generar_grafo(degree_seq)
        self.grafo_dict = self.diccionario_grafo(self.grafo)
        self.num_vertices = len(self.grafo.nodes())

    def generar_grafo(self, degree_seq):
        # Suma de grados debe ser par - se debe cumplir "Handshaking Lemma"
        if sum(degree_seq) % 2 != 0:
            raise ValueError("Suma de grados no válida, la suma de todos los grados debe ser par...")

        G = nx.configuration_model(degree_seq)
        # Hacerlo un grafo simple - eliminar loops y aristas paralelas
        G = nx.Graph(G)
        G.remove_edges_from(nx.selfloop_edges(G))

        return G

    def diccionario_grafo(self, G):
        grafo_dict = {}

        colores_disponibles = list(range(1, self.num_min_colores + 1))  # Del 1 al 4
        shuffle(colores_disponibles)  # Mezclar colores

        for vertice in G.nodes():
            adyacentes = set(G.neighbors(vertice)) # Conjunto de vértices adyacentes
            if len(colores_disponibles) > 0:       # Asegurar que se usen los 4 colores
                color = colores_disponibles.pop()
            else:
                color = random.randint(1, self.num_min_colores)  # Asignar aleatorios (1 al 4) para los que faltan
            grafo_dict[vertice] = (adyacentes, color, 0)     # Inicializar conflictos en 0

        self.contar_conflictos(grafo_dict)

        return grafo_dict

    def contar_conflictos(self, grafo_dict):
      for vertice in grafo_dict:
          vecinos = grafo_dict[vertice][0]
          color = grafo_dict[vertice][1]

          # Resetear conflictos antes de recalcular
          conflictos = 0
          for vecino in vecinos:
              color_vecino = grafo_dict[vecino][1]

              if color_vecino == color:
                  conflictos += 1

          grafo_dict[vertice] = (vecinos, color, conflictos)

    def calcular_conflictos_totales(self):
      suma_conflictos = 0

      for vertice in self.grafo_dict:
          conflictos = self.grafo_dict[vertice][2]
          suma_conflictos += conflictos  # Acumular los conflictos

      return suma_conflictos
    def imprimir_dict_grafo(self):
        print("Diccionario del grafo:")
        for i, (k, v) in enumerate(self.grafo_dict.items()):
            print(f"{k}: {v}", end="  ")
            if (i + 1) % 1 == 0:
                print()
        print()

    def plot_grafo(self):
        plt.figure(figsize=(6,6))
        pos = nx.spring_layout(self.grafo)
        color_vertice = []

        for vertice in self.grafo.nodes():
            color = self.grafo_dict[vertice][1]
            color_vertice.append(color)

        nx.draw(self.grafo, pos, with_labels=True, node_color=color_vertice, cmap=plt.cm.viridis, edge_color='gray', node_size=700, font_size=12, font_color='white')
        plt.title("Coloreado del Grafo")
        plt.show()

In [653]:
# @title **Secuencias válidas para generar el grafo**

def generar_secuencia_grados(num_vertices, min_grado, max_grado):
    grados = []

    for _ in range(num_vertices):
        grado_aleatorio = random.randint(min_grado, max_grado)
        grados.append(grado_aleatorio)

    suma_grados = sum(grados)
    if suma_grados % 2 != 0:  # Hacer par si es impar
        grados[-1] -= 1

    return grados

#degree_seq = generar_secuencia_grados(10, 2, 6)
#print("Usar secuencia:", degree_seq)

In [654]:
# @title **Cambio de temperatura - Lineal Exponencial**
def cambio_de_temperatura(temperatura_actual,opcion='lineal',factor_enfriamiento=0.98):
    if opcion == 'lineal':
        return temperatura_actual*(1-(0.951-factor_enfriamiento))
    elif opcion == 'exponencial':
        return temperatura_actual * factor_enfriamiento

def get_curva(temperatura_actual=1000,opcion='lineal',factor_enfriamiento=0.98,delta=1):
    nueva_temp=[]
    prob_ac = []
    temp_act = temperatura_actual
    for i in range(100):
      temp_act = cambio_de_temperatura(temp_act,opcion,factor_enfriamiento)
      nueva_temp.append(temp_act)
      prob_ac.append(math.exp(-delta / temp_act))
    return nueva_temp,prob_ac

temp, proba = get_curva(factor_enfriamiento=0.98)
print(temp[-1])

temp, proba = get_curva(opcion='exponencial',factor_enfriamiento=0.98)
print(temp[-1])

17439.638415178048
132.61955589475298


In [671]:
# @title **Recocido Simulado**
class SimulatedAnealingCG(ColoreadoGrafo):
    def __init__(self, degree_seq, num_min_colores = 4, max_iter = 1000, temp_ini = 2000):
        super().__init__(degree_seq, num_min_colores)
        self.max_iteraciones = max_iter
        self.temperatura_inicial = temp_ini
        self.opcion = 'exponencial'
        self.factor_enfriamiento = 0.98

    def set_parametros(self, temp = 1000, opcion = 'exponencial', fe = 0.98, max_it = 1000):
        self.temperatura_inicial = temp
        self.opcion = opcion
        self.factor_enfriamiento = fe
        self.max_iteraciones = max_it

    def cambio_de_temperatura(self, temperatura_actual):
        if self.opcion == 'lineal':
            return temperatura_actual * (1 - (0.951 - self.factor_enfriamiento))
        elif self.opcion == 'exponencial':
            return temperatura_actual * self.factor_enfriamiento

    def crear_vecino(self):
      nodos_con_conflicto = []

      for vertice in self.grafo_dict:
          if self.grafo_dict[vertice][2] > 0:
              nodos_con_conflicto.append(vertice)

      if len(nodos_con_conflicto) > 0:
          vertice = random.choice(nodos_con_conflicto)
      else:
          vertice = random.choice(list(self.grafo_dict.keys()))

      vecinos = self.grafo_dict[vertice][0]
      color_actual = self.grafo_dict[vertice][1]

      colores_vecinos = set()  # no repetir colores

      for vecino in vecinos:
          color_vecino = self.grafo_dict[vecino][1]
          colores_vecinos.add(color_vecino)

      colores_disponibles = []

      for color in range(1, self.num_min_colores + 1):
          if color != color_actual and color not in colores_vecinos:
              colores_disponibles.append(color)

      # cambiar el color del nodo si hay colores disponibles
      if len(colores_disponibles) > 0:
          nuevo_color = random.choice(colores_disponibles)
          self.grafo_dict[vertice] = (vecinos, nuevo_color, 0)

          self.contar_conflictos(self.grafo_dict)  # Volver a contar conflictos

    def recocido_simulado(self):
        temperatura_actual = self.temperatura_inicial
        iteraciones = 0
        costo_actual = self.calcular_conflictos_totales()  # del grafo inicial

        while temperatura_actual > 0.1 and iteraciones < self.max_iteraciones:
            self.crear_vecino()
            costo_vecino = self.calcular_conflictos_totales()

            delta_costo = costo_vecino - costo_actual
            if delta_costo < 0 or random.random() < math.exp(-delta_costo / temperatura_actual):
                costo_actual = costo_vecino

            temperatura_actual = self.cambio_de_temperatura(temperatura_actual)
            iteraciones += 1
            print(f"Iteración {iteraciones}: Costo actual: {costo_actual}, Temperatura: {temperatura_actual:.2f}")

        return self.grafo_dict, costo_actual

degree_seq = generar_secuencia_grados(100, 3, 4)
grafo_simulado = SimulatedAnealingCG(degree_seq, num_min_colores=4)
grafo_coloreado, costo_final = grafo_simulado.recocido_simulado()

Iteración 1: Costo actual: 90, Temperatura: 1960.00
Iteración 2: Costo actual: 84, Temperatura: 1920.80
Iteración 3: Costo actual: 80, Temperatura: 1882.38
Iteración 4: Costo actual: 78, Temperatura: 1844.74
Iteración 5: Costo actual: 76, Temperatura: 1807.84
Iteración 6: Costo actual: 74, Temperatura: 1771.68
Iteración 7: Costo actual: 72, Temperatura: 1736.25
Iteración 8: Costo actual: 70, Temperatura: 1701.53
Iteración 9: Costo actual: 68, Temperatura: 1667.50
Iteración 10: Costo actual: 64, Temperatura: 1634.15
Iteración 11: Costo actual: 62, Temperatura: 1601.46
Iteración 12: Costo actual: 60, Temperatura: 1569.43
Iteración 13: Costo actual: 58, Temperatura: 1538.04
Iteración 14: Costo actual: 56, Temperatura: 1507.28
Iteración 15: Costo actual: 54, Temperatura: 1477.14
Iteración 16: Costo actual: 50, Temperatura: 1447.60
Iteración 17: Costo actual: 46, Temperatura: 1418.64
Iteración 18: Costo actual: 44, Temperatura: 1390.27
Iteración 19: Costo actual: 40, Temperatura: 1362.47
It

In [672]:
# @title **Resultado**
grafo_simulado.imprimir_dict_grafo()
print("Costo final:", costo_final)
#grafo_simulado.plot_grafo()

Diccionario del grafo:
0: ({1, 84, 13}, 4, 0)  
1: ({0, 68, 29, 70}, 3, 0)  
2: ({58, 20, 71, 23}, 1, 0)  
3: ({8, 42, 68, 78}, 3, 0)  
4: ({25, 6, 15}, 2, 0)  
5: ({57, 91, 52}, 3, 0)  
6: ({54, 9, 4, 94}, 4, 0)  
7: ({32, 98, 99, 62}, 1, 0)  
8: ({66, 3, 60, 46}, 1, 0)  
9: ({27, 19, 6}, 3, 0)  
10: ({96, 93, 14}, 2, 0)  
11: ({64, 51, 22}, 1, 0)  
12: ({97, 76, 68, 31}, 1, 0)  
13: ({0, 58, 27, 90}, 2, 0)  
14: ({10, 39, 87}, 3, 0)  
15: ({72, 51, 4}, 1, 0)  
16: ({28, 61, 62}, 2, 0)  
17: ({42, 26, 61, 22}, 2, 0)  
18: ({88, 40, 67, 85}, 4, 0)  
19: ({9, 33}, 4, 0)  
20: ({24, 2, 31}, 2, 0)  
21: ({89, 44, 78}, 3, 0)  
22: ({17, 50, 11, 79}, 4, 0)  
23: ({72, 81, 2, 31}, 2, 0)  
24: ({74, 20, 66, 36}, 3, 0)  
25: ({64, 4, 29}, 4, 0)  
26: ({56, 17, 67, 85}, 3, 0)  
27: ({9, 77, 13, 60}, 1, 0)  
28: ({16, 91, 95}, 3, 0)  
29: ({48, 1, 25}, 1, 0)  
30: ({81, 98, 52}, 3, 0)  
31: ({75, 12, 20, 23}, 3, 0)  
32: ({42, 43, 7}, 3, 0)  
33: ({41, 19, 35}, 3, 0)  
34: ({80, 65, 43}, 2, 0)  